# [STARTER] Exercise - Create an Agent with external API call enabled

In this exercise, you'll build an agent that can interact with external APIs to gather real-time data
and provide responses based on that information. You'll combine concepts from state management and
memory while adding the ability to make external API calls safely and effectively.


## Challenge

Your task is to create an agent that can make External API Calls:

- Implement tools that interact with real APIs
- Handle API responses and errors gracefully
- Use environment variables for API keys
- Process and format API data for user consumption

## Setup
First, let's import the necessary libraries:

In [1]:
from pathlib import Path
import sys

nb_dir = Path(__file__).parent if "__file__" in globals() else Path.cwd()
parent = nb_dir.parent  # points to 03-building-agents
if str(parent) not in sys.path:
    sys.path.insert(0, str(parent))

In [2]:
import os
from typing import List
import requests
from dotenv import load_dotenv

from lib.agents import Agent
from lib.messages import BaseMessage
from lib.tooling import tool
from lib.tools import get_all_tools

In [3]:
load_dotenv()

False

## Define API tools

Feel free to use any open service available through APIs.

Here are a few examples. You can follow the instructions given.
- https://jsonplaceholder.typicode.com/guide/
- https://www.exchangerate-api.com/
- https://openweathermap.org/

Or you can find one you're interested in here.
- https://github.com/public-apis/public-apis

In [4]:
# TODO: Add all the tools you have defined
tools = get_all_tools()

def build_instructions_with_tools(tools) -> str:
    # Build a concise, self-updating Tools section from the tool objects
    tool_lines = []
    for t in tools:
        sig = str(getattr(t, "signature", ""))  # e.g. (city: str) -> OpenWeatherResponse
        desc = getattr(t, "description", "") or ""
        tool_lines.append(f"- {t.name}{sig}: {desc}")
    tools_catalog = "\n".join(tool_lines)

    return (
        "You are a helpful assistant with the following capabilities:\n"
        "\n"
        "Behavioral guidelines:\n"
        "- Ask for missing parameters before calling a tool.\n"
        "- Use a tool only when it will improve factual accuracy.\n"
        "- If a tool call fails or returns partial data, report what you do know and what’s missing.\n"
        "- Keep answers concise and actionable. Include units and currency codes where relevant.\n"
        "- When applicable, show a short, single-step calculation (e.g., 100 USD to BRL) using the returned rate.\n"
        "\n"
        "Tools you can use:\n"
        f"{tools_catalog}\n"
        "\n"
        "Formatting:\n"
        "- Weather: city, conditions, temperature in °C, wind speed.\n"
        "- Exchange rate: base→target, rate (4 decimals), optional 100-unit example.\n"
        "- Posts: summarize title/body briefly; include post id.\n"
    )

In [5]:
# TODO: Add instructions to your agent
instructions = build_instructions_with_tools(tools)

agent = Agent(
    model_name="gpt-4o-mini",
    instructions=instructions,
    tools=tools,
)

In [6]:
def print_messages(messages: List[BaseMessage]):
    for m in messages:
        print(f" -> (role = {m.role}, content = {m.content}, tool_calls = {getattr(m, 'tool_calls', None)})")

## Run your Agent

In [7]:
# TODO: Change the query and then run your agent
query = "What’s the current weather in São Paulo and convert 100 USD to BRL? Show the rate (4 decimals) and the calculation."
session_id = "external_tools"

In [8]:
run1 = agent.invoke(
    query=query, 
    session_id=session_id,
)

print("\nMessages from run 1:")
messages = run1.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 1:
 -> (role = system, content = You are a helpful assistant with the following capabilities:

Behavioral guidelines:
- Ask for missing parameters before calling a tool.
- Use a tool only when it will improve factual accuracy.
- If a tool call fails or returns partial data, report what you do know and what’s missing.
- Keep answers concise and actionable. Include units and currency codes where relevant.
- When applicable, show a short, single-step calculation (e.g., 100 USD to BRL) using the returned rate.

Tools you can use:
- get_got_quote() -> lib.tools.GOTQuote: Get a random Game of Thrones quote.

Inputs:
    None

Returns:
    GOTQuote: {"sentence": str, "character": {"name": str, "slug": str, "house": {"nam

In [9]:
run2 = agent.invoke(
    query="Tell me a quote from Game of Thrones", 
    session_id=session_id,
)

print("\nMessages from run 1:")
messages = run2.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 1:
 -> (role = system, content = You are a helpful assistant with the following capabilities:

Behavioral guidelines:
- Ask for missing parameters before calling a tool.
- Use a tool only when it will improve factual accuracy.
- If a tool call fails or returns partial data, report what you do know and what’s missing.
- Keep answers concise and actionable. Include units and currency codes where relevant.
- When applicable, show a short, single-step calculation (e.g., 100 USD to BRL) using the returned rate.

Tools you can use:
- get_got_quote() -> lib.tools.GOTQuote: Get a random Game of Thrones quote.

Inputs:
    None

Returns:
    GOTQuote: {"sentence": str, "character": {"name": str, "slug": str, "house": {"nam

In [10]:
run3 = agent.invoke(
    query="Combine the last two messages into a single clever one", 
    session_id=session_id,
)

print("\nMessages from run 3:")
messages = run3.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 3:
 -> (role = system, content = You are a helpful assistant with the following capabilities:

Behavioral guidelines:
- Ask for missing parameters before calling a tool.
- Use a tool only when it will improve factual accuracy.
- If a tool call fails or returns partial data, report what you do know and what’s missing.
- Keep answers concise and actionable. Include units and currency codes where relevant.
- When applicable, show a short, single-step calculation (e.g., 100 USD to BRL) using the returned rate.

Tools you can use:
- get_got_quote() -> lib.tools.GOTQuote: Get a random Game of Thrones quote.

Inputs:
    None

Returns:
    GOTQuote: {"sentence": str, "character": {"name": str, "slug": str, "house": {"name": str, ...}}}

Notes:
    - Source: `https://api.gameofthronesquotes.xyz/v1/random`
    

## Check session histories

In [11]:
runs = agent.get_session_runs(session_id)
for i, run_object in enumerate(runs, 1):
    print(f"\n# Run {i}", run_object.metadata)
    print("Messages:")
    print_messages(run_object.get_final_state()["messages"])


# Run 1 {'run_id': 'f24cb31a-fae2-4ac9-abbb-1822a2725e38', 'start_timestamp': '2025-10-11 16:30:39.238972', 'end_timestamp': '2025-10-11 16:30:46.521654', 'snapshot_counts': 5}
Messages:
 -> (role = system, content = You are a helpful assistant with the following capabilities:

Behavioral guidelines:
- Ask for missing parameters before calling a tool.
- Use a tool only when it will improve factual accuracy.
- If a tool call fails or returns partial data, report what you do know and what’s missing.
- Keep answers concise and actionable. Include units and currency codes where relevant.
- When applicable, show a short, single-step calculation (e.g., 100 USD to BRL) using the returned rate.

Tools you can use:
- get_got_quote() -> lib.tools.GOTQuote: Get a random Game of Thrones quote.

Inputs:
    None

Returns:
    GOTQuote: {"sentence": str, "character": {"name": str, "slug": str, "house": {"name": str, ...}}}

Notes:
    - Source: `https://api.gameofthronesquotes.xyz/v1/random`
    - 

In [12]:
runs = agent.get_session_runs(session_id)
for run_object in runs:
    print(run_object)
    for snp in run_object.snapshots:
        print(f"-> {snp}")
    print("\n")

Run('f24cb31a-fae2-4ac9-abbb-1822a2725e38')
-> Snapshot('ddb18126-e276-4996-b1c7-7fc689764c2e') @ [2025-10-11 16:30:39.239313]: __entry__.State({'user_query': 'What’s the current weather in São Paulo and convert 100 USD to BRL? Show the rate (4 decimals) and the calculation.', 'instructions': 'You are a helpful assistant with the following capabilities:\n\nBehavioral guidelines:\n- Ask for missing parameters before calling a tool.\n- Use a tool only when it will improve factual accuracy.\n- If a tool call fails or returns partial data, report what you do know and what’s missing.\n- Keep answers concise and actionable. Include units and currency codes where relevant.\n- When applicable, show a short, single-step calculation (e.g., 100 USD to BRL) using the returned rate.\n\nTools you can use:\n- get_got_quote() -> lib.tools.GOTQuote: Get a random Game of Thrones quote.\n\nInputs:\n    None\n\nReturns:\n    GOTQuote: {"sentence": str, "character": {"name": str, "slug": str, "house": {"na